<h1 align="center"> Capacitive ADC High-Level Modelling </h1>

<h2 align="left"> Successive Approximation Register (SAR) ADC - Bottom Plate Sampling </h2>

<p align="justify">
    The SAR ADC is a type of ADC that uses a binary search algorithm to find the input voltage. The SAR ADC is composed of a DAC, a comparator, and a SAR control logic and register. The binary search divides the search domain in 2 in each conversion step, making the input voltage of the comparator to approximate the input voltage of the ADC in each step, by reprogramming each of the bits of the intrinsic DAC to uncover which is the digital word that best represents vin.
</p>

INSERIR IMAGEM DE UM SAR ADC CLÁSSICO

INSERIR IMAGEM DA OPERAÇÃO DE DIVISÃO DO DOMÍNIO DE BUSCA

INSERIR IMAGEM DA CONFIGURAÇÃO DO ADC EM CADA PASSO DE CONVERSÃO

<h2 align="left"> Successive Approximation Register (SAR) ADC - Top Plate Sampling </h2>

<p align="justify">
    
</p>

INSERIR IMAGEM DE UM SAR ADC CLÁSSICO com top plate sampling

INSERIR IMAGEM DA OPERAÇÃO DE DIVISÃO DO DOMÍNIO DE BUSCA

INSERIR IMAGEM DA CONFIGURAÇÃO DO ADC EM CADA PASSO DE CONVERSÃO

<h2 align="left"> Differential Successive Approximation Register (SAR) ADC - Bottom Plate Sampling </h2>

<p align="justify">
    
</p>

INSERIR IMAGEM DE UM SAR ADC DIFERENCIAL COM BOT PLATE SAMPLING

INSERIR IMAGEM DA OPERAÇÃO DE DIVISÃO DO DOMÍNIO DE BUSCA E EXPLICAR O PORQUE DE NAO PRECISAR DE USAR O DAC PARA DETERMINAR O PRIMEIRO BIT

INSERIR IMAGEM DA CONFIGURAÇÃO DO ADC EM CADA PASSO DE CONVERSÃO, E EXPLICAR O PORQUE DE NECESSITAR DE UM PASSO ADICIONAL PARA PASSAR O VIN DA BOT PLATE PARA A TOP PLATE

<h2 align="left"> Differential Successive Approximation Register (SAR) ADC - Top Plate Sampling </h2>

<p align="justify">
    
</p>

INSERIR IMAGEM DE UM SAR ADC DIFERENCIAL COM TOP PLATE SAMPLING

INSERIR IMAGEM DA OPERAÇÃO DE DIVISÃO DO DOMÍNIO DE BUSCA E EXPLICAR O PORQUE DE NAO PRECISAR DE USAR O DAC PARA DETERMINAR O PRIMEIRO BIT

INSERIR IMAGEM DA CONFIGURAÇÃO DO ADC EM CADA PASSO DE CONVERSÃO

<h2 align="left"> Successive Approximation Register (SAR) ADC - Top Plate Sampling & Monotonic SAR</h2>

<p align="justify">

</p>

INSERIR IMAGEM DE UM SAR ADC DIFEERENCIAL COM TOP PLATE SAMPLING E MONOTONICO

INSERIR IMAGEM DA OPERAÇÃO DE DIVISÃO DO DOMÍNIO DE BUSCA COM O ALGORITMO MONOTONICO

INSERIR IMAGEM DA CONFIGURAÇÃO DO ADC EM CADA PASSO DE CONVERSÃO

<h2 align="left"> Successive Approximation Register (SAR) ADC - V<sub>CM</sub>-Based with Bottom Plate Sampling   </h2>

<p align="justify">
    
</p>


<h2 align="left"> Differential Successive Approximation Register (SAR) ADC - Loop Unrolled </h2>

<p align="justify">
    
</p>

INSERIR IMAGEM DE UM SAR ADC DIFERENCIAL COM UM COMPARADOR POR BIT

INSERIR IMAGEM DA OPERAÇÃO DE DIVISÃO DO DOMÍNIO DE BUSCA DO SAR ADC LOP-UNROLLED

INSERIR IMAGEM DA CONFIGURAÇÃO DO ADC EM CADA PASSO DE CONVERSÃO

EXPLICAR COMO O PING-PONG SAR PODE SER DERIVADO DO LOOP-UNROLLED SAR ADC E EXPLICAR O SEU FUNCIONAMENTO, BEM COMO O ESQUEMA DE COMUTACAO

In [ ]:
# MODELAR O PING PONG SAR ADC